In [1]:
import mygene
import pandas as pd
import numpy as np

In [2]:
mg = mygene.MyGeneInfo()

In [12]:
df_tr = pd.read_csv("nsFilterDflt_v0_MAV-G_498_ALL_tr.txt", sep='\t', index_col=0, header=0)
df_ts = pd.read_csv("nsFilterDflt_v0_MAV-G_498_ALL_ts.txt", sep='\t', index_col=0, header=0)

In [13]:
np.all(df_tr.columns == df_ts.columns)

True

In [4]:
xli = df_tr.columns.values.tolist()

In [ ]:
# out = mg.querymany(xli, scopes='entrezgene', fields='symbol', species='human')

In [5]:
out = mg.getgenes(xli, fields='symbol,name', as_dataframe=True, species='human')

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-12464...done.


In [6]:
out.shape

(12464, 5)

In [7]:
# Discard Entrez IDs without Gene symbol
idx = np.where(out['notfound'].isna())[0]

In [8]:
idx.shape

(12304,)

In [16]:
genes = out.iloc[idx, :].drop(['_score', 'name', 'notfound'], axis=1)

In [14]:
# Slice df to resulting genes
df_tr = df_tr.iloc[:, idx]
df_ts = df_ts.iloc[:, idx]

In [17]:
df_tr.columns = genes['symbol']
df_ts.columns = genes['symbol']

In [18]:
df_tr.head()

symbol,NAT2,ADA,POU5F1P6,NR2E3,NAALAD2,VN2R10P,SNORD116-1,SNORD116-13,SNORD116-14,SNORD116-16,...,HS3ST2,HS3ST1,MVP,FGF19,TNFSF15,NR1H4,CLEC2B,CDC42,CDH1,KCNE2
sampleID,,,,,,,,,,,,,,,,,,,,,
NB001,0.00,13.01,3.61,4.48,12.56,8.23,9.49,2.97,0.00,5.90,...,9.81,10.55,14.67,11.26,9.20,5.48,12.20,8.60,11.54,0.00
NB003,0.00,11.22,0.00,7.35,13.43,4.11,10.63,7.09,5.94,3.86,...,8.89,11.04,12.91,0.00,4.77,0.00,9.68,8.90,7.22,3.95
NB005,7.71,12.24,6.18,7.17,13.43,0.00,11.60,9.15,4.11,0.00,...,9.29,10.24,14.58,0.00,9.80,0.00,9.77,9.09,6.81,0.00
NB007,5.57,13.56,3.72,7.73,12.47,6.89,0.00,8.29,0.00,5.83,...,10.31,9.72,15.27,8.84,10.12,0.00,12.68,9.91,10.10,4.16
NB009,7.57,11.71,3.80,5.71,14.57,6.99,9.81,8.38,0.00,0.00,...,10.20,10.09,13.95,0.00,7.31,6.14,11.02,8.84,8.43,5.86


In [19]:
df_ts.head()

symbol,NAT2,ADA,POU5F1P6,NR2E3,NAALAD2,VN2R10P,SNORD116-1,SNORD116-13,SNORD116-14,SNORD116-16,...,HS3ST2,HS3ST1,MVP,FGF19,TNFSF15,NR1H4,CLEC2B,CDC42,CDH1,KCNE2
sampleID,,,,,,,,,,,,,,,,,,,,,
NB002,0.00,12.39,0.00,5.61,12.94,0.00,10.84,8.17,4.24,0.00,...,9.35,8.86,13.50,9.37,9.03,0.00,10.46,9.35,9.14,0.00
NB004,6.98,11.86,7.32,8.75,14.04,6.89,11.21,10.12,3.89,3.84,...,10.20,12.05,13.99,5.60,7.87,11.15,10.55,8.70,8.39,4.21
NB006,2.48,11.37,0.00,4.23,14.65,8.00,10.95,9.50,0.00,7.22,...,7.33,9.52,13.19,7.18,7.49,0.00,9.60,8.85,9.11,0.00
NB008,0.00,11.23,0.00,5.61,14.70,7.98,11.59,8.96,0.00,7.06,...,10.12,8.43,12.76,3.00,7.90,0.00,9.59,7.96,7.10,0.00
NB010,0.00,10.06,3.62,7.00,13.87,4.10,11.09,3.14,0.00,0.00,...,9.86,10.06,13.39,0.00,5.83,0.00,9.20,8.51,8.19,0.00


Now add 'class' as the first column:

In [45]:
# read previous dataframes, which contained the class
tmp = pd.read_csv("nb_train_old.txt.gz", sep='\t', header=0, index_col=0)
y_tr = pd.DataFrame(tmp['class'])

In [46]:
tmp = pd.read_csv("nb_test_old.txt.gz", sep='\t', header=0, index_col=0)
y_ts = pd.DataFrame(tmp['class'])

In [47]:
# join with new dataframes
df_tr = y_tr.join(df_tr)
df_ts = y_ts.join(df_ts)

In [48]:
df_tr.to_csv("nb_train.txt.gz", sep='\t', compression="gzip")
df_ts.to_csv("nb_test.txt.gz", sep='\t', compression="gzip")